<div class="alert alert-warning">
    
<b>Disclaimer:</b> 
    
The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function  
- setting and run the fit  
- extracting and displaying information about the results  

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.

</div>


# Water Teixeira &lowast; Resolution with bumps (2 wavelengths)

## Table of Contents

- [Introduction](#Introduction)
- [Import and install required libraries](#Import-and-install-required-libraries) 
- [Setting of fitting](#Setting-of-fitting)  
- [Running the fit](#Running-the-fit) 
- [Showing the results](#Showing-the-results) 

[Top](#Table-of-Contents)

## Introduction

<div class="alert alert-info">
    
The objective of this notebook is to show how to use the <b>water_teixeira</b> model and fit the data using bumps.
</div>

The data are two sets of water data measured at IN5 (ILL) using two different wavelengths, 5 and 8 &#x212B;.

**Reference:** J. Qvist, H. Schober and B. Halle, *J. Chem. Phys.* **134**, 144508 (2011)

[Top](#Table-of-Contents)

## Import and install required libraries

In [ ]:
from __future__ import print_function

In [ ]:
# the following two lines are to remove the warning about too many figures open simultaneously
from matplotlib import rcParams
rcParams.update({'figure.max_open_warning': 0})

import sys
import os
import ipywidgets

In [ ]:
# install bumps (if not already installed)
import pkgutil
if not pkgutil.find_loader("bumps"):   
    bumpsY = ipywidgets.Button(description='Yes', button_style='success')
    bumpsN = ipywidgets.Button(description='No', button_style='danger')
    choice_installation = ipywidgets.VBox(
    [ipywidgets.Label("Do you want to install bumps?"), ipywidgets.HBox([bumpsY, bumpsN])],
    layout=ipywidgets.Layout(width='30%', height='80px'))

    display(choice_installation)
    
    def on_bumpsY_clicked(b):
        !{sys.executable} -m pip install bumps==0.7.12
    def on_bumpsN_clicked(b):
        print("You will not be able to run some of the remaining parts of this notebook")
        
    bumpsY.on_click(on_bumpsY_clicked)
    bumpsN.on_click(on_bumpsN_clicked)

In [ ]:
import bumps

# check version of bumps installed
# Information message if installed version not recent enough
from distutils.version import StrictVersion
if StrictVersion(bumps.__version__) <= StrictVersion('0.7.6'):
    print("""The version of bumps installed is not recent 
    enough to run the examples. 
    Please update bumps. The minimum version required is 0.7.8""")

[Top](#Table-of-Contents)

## Setting of fitting 

### install QENSmodels (if not already installed)

In [ ]:
if not pkgutil.find_loader("QENSmodels"):
    buttonY = ipywidgets.Button(description='Yes', button_style='success')
    buttonN = ipywidgets.Button(description='No', button_style='danger')
    choice_installation = ipywidgets.VBox(
    [ipywidgets.Label("Do you want to install the QENSmodels' library?"), ipywidgets.HBox([buttonY, buttonN])],
    layout=ipywidgets.Layout(width='50%', height='80px'))
    display(choice_installation)
    
    def on_buttonY_clicked(b):
        !{sys.executable} -m pip install git+https://github.com/QENSlibrary/QENSmodels#egg=QENSmodels

    def on_buttonN_clicked(b):
        print("You will not be able to run some of the remaining parts of this notebook")
     
    buttonY.on_click(on_buttonY_clicked)
    buttonN.on_click(on_buttonN_clicked) 

### import reference data

In [ ]:
import h5py
import QENSmodels
import numpy as np
from scipy.integrate import simps
import bumps.names as bmp
from bumps.fitters import fit
from bumps.formatnum import format_uncertainty

path_to_data = './data/'

# Data
# Wavelength 5 Angstrom
with h5py.File(path_to_data + 'H2O_293K_5A.hdf', 'r') as f:
    hw_5A = f['entry1']['data1']['X'][:]
    q_5A = f['entry1']['data1']['Y'][:]
    unit_w5A = f['entry1']['data1']['X'].attrs['long_name']
    unit_q5A = f['entry1']['data1']['Y'].attrs['long_name']
    sqw_5A = np.transpose(f['entry1']['data1']['DATA'][:])
    err_5A = np.transpose(f['entry1']['data1']['errors'][:])


# Wavelength 8 Angstrom
with h5py.File(path_to_data + 'H2O_293K_8A.hdf', 'r') as f:
    hw_8A = f['entry1']['data1']['X'][:]
    q_8A = f['entry1']['data1']['Y'][:]
    unit_w8A = f['entry1']['data1']['X'].attrs['long_name']
    unit_q8A = f['entry1']['data1']['Y'].attrs['long_name']
    sqw_8A = np.transpose(f['entry1']['data1']['DATA'][:])
    err_8A = np.transpose(f['entry1']['data1']['errors'][:])

# Resolution
# Wavelength 5 Angstrom
with h5py.File(path_to_data + 'V_273K_5A.hdf', 'r') as f:
    res_5A = np.transpose(f['entry1']['data1']['DATA'][:])

# Wavelength 8 Angstrom
with h5py.File(path_to_data + 'V_273K_8A.hdf', 'r') as f:
    res_8A = np.transpose(f['entry1']['data1']['DATA'][:])

# Force resolution function to have unit area
# 5 Angstrom
for i in range(len(q_5A)):
    area = simps(res_5A[:, i], hw_5A)
    res_5A[:, i] /= area

# 8 Angstrom
for i in range(len(q_8A)):
    area = simps(res_8A[:, i], hw_8A)
    res_8A[:, i] /= area

# Fit range -1 to +1 meV
idx_5A = np.where(np.logical_and(hw_5A > -1.0, hw_5A < 1.0))
idx_8A = np.where(np.logical_and(hw_8A > -1.0, hw_8A < 1.0))


def model_convol(x, q, scale=1, center=0, D=1, resTime=1, radius=1, DR=1, resolution=None):
    model = QENSmodels.sqwWaterTeixeira(x, q, scale, center, D, resTime, radius, DR)
    return np.convolve(model, resolution / resolution.sum(), mode='same')

### Physical units
For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data
dict_physical_units = {'omega': "1/ps",
                       'q': "1/Angstrom",
                       'scale': "unit_of_signal/ps",
                       'center': "1/ps",
                       'D': "Angstrom^2/ps",
                       'radius': "Angstrom",
                       'resTime': "ps"}

### display units of input data 
Just for information in order to determine if a convertion of units is required before using the QENSmodels

In [ ]:
print(f"""At 5 Angstroms, the names and units of `w` ( `x`axis) and `q` are: 
{str(unit_w5A[0], 'utf-8')} and {str(unit_q5A[0], 'utf-8')}, respectively.""")

print(f"""At 8 Angstroms, the names and units of `w` ( `x`axis) and `q` are: 
{str(unit_w8A[0], 'utf-8')} and {str(unit_q8A[0], 'utf-8')}, respectively.""")

### create fitting model

In [ ]:
# Fit
M = []

# First dataset: wavelength=5 Angstrom
for i in range(len(q_5A)):

    x = hw_5A[idx_5A]
    data = sqw_5A[idx_5A, i]
    error = err_5A[idx_5A, i]
    resol = res_5A[idx_5A, i]

    # Select only valid data (error = -1 for Q, w points not accessible)
    valid = np.where(error > 0.0)
    x = x[valid[1]]
    data = data[valid]
    error = error[valid]
    resol = resol[valid]

    # Teixeira model
    Mq = bmp.Curve(model_convol,
                   x,
                   data,
                   error,
                   q=q_5A[i],
                   scale=10,
                   center=0.0,
                   D=0.13,
                   resTime=0.1,
                   radius=1.0,
                   DR=0.3,
                   resolution=resol)

    # Fitted parameters
    Mq.scale.range(0, 1e2)
    Mq.center.range(-0.1, 0.1)
    Mq.D.range(0.05, 0.25)
    Mq.resTime.range(0, 1)
    Mq.radius.range(0.9, 1.1)
    Mq.DR.range(0, 1)

    # Q-independent parameters
    if i == 0:
        QD = Mq.D
        QT = Mq.resTime
        QR = Mq.radius
        QDR = Mq.DR
    else:
        Mq.D = QD
        Mq.resTime = QT
        Mq.radius = QR
        Mq.DR = QDR

    M.append(Mq)
    
# Second dataset: wavelength=8 Angstrom
for i in range(len(q_8A)):

    x = hw_8A[idx_8A]
    data = sqw_8A[idx_8A, i]
    error = err_8A[idx_8A, i]
    resol = res_8A[idx_8A, i]

    # Select only valid data (error = -1 for Q, w points not accessible)
    valid = np.where(error > 0.0)
    x = x[valid[1]]
    data = data[valid]
    error = error[valid]
    resol = resol[valid]

    Mq = bmp.Curve(model_convol,
                   x,
                   data,
                   error,
                   q=q_8A[i],
                   scale=10,
                   center=0.0,
                   D=0.13,
                   resTime=0.1,
                   radius=1.0,
                   DR=0.3,
                   resolution=resol)

    # Fitted parameters
    Mq.scale.range(0, 1e2)
    Mq.center.range(-0.1, 0.1)
    Mq.D.range(0.05, 0.25)
    Mq.resTime.range(0, 1)
    Mq.radius.range(0.9, 1.1)
    Mq.DR.range(0, 1)

    # Q-independent parameters set with 5A data
    Mq.D = QD
    Mq.resTime = QT
    Mq.radius = QR
    Mq.DR = QDR

    M.append(Mq)

problem = bmp.FitProblem(M)

### Choice of minimizer for bumps

In [ ]:
options_dict={'Levenberg-Marquardt': "lm", 
             'Nelder-Mead Simplex': "amoeba", 
             'DREAM': "dream", 
             'Differential Evolution': "de", 
             'Quasi-Newton BFGS': "newton", 
             'Random Lines (experimental)': "rl", 
             'Particle Swarm (experimental)': "ps", 
             'Parallel Tempering (experimental)': "pt"}

w_choice_minimizer = ipywidgets.Dropdown(
    options=list(options_dict.keys()),
    value='Levenberg-Marquardt',
    description='Minimizer:',
    layout=ipywidgets.Layout(height='40px')
)
w_choice_minimizer

### Setting for running bumps 

In [ ]:
steps_fitting = ipywidgets.IntText(
    value=100,
    description='Number of steps when fitting',
    style={'description_width': 'initial'})

steps_fitting

In [ ]:
# Preview of the settings
print('Initial chisq', problem.chisq_str())

In [ ]:
problem.plot()

In [ ]:
problem.show()

[Top](#Table-of-Contents)

## Running the fit

Run the fit using the *minimizer* defined above with a number of *steps* also specified above.

In [ ]:
result = fit(problem,
             method=options_dict[w_choice_minimizer.value],
             steps=int(steps_fitting.value),
             verbose=True)

[Top](#Table-of-Contents)

## Showing the results

In [ ]:
problem.plot()

In [ ]:
# Display fitting results
problem.show()

In [ ]:
# Print chi**2 and parameters' values after fit
print("final chisq", problem.chisq_str())
for k, v, dv in zip(problem.labels(), result.x, result.dx):
    if k in dict_physical_units.keys():
        print(k, ":", format_uncertainty(v, dv), dict_physical_units[k])
    else:
        print(k, ":", format_uncertainty(v, dv))